In [5]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [6]:
gid = 0
srv = 1
c = 0
cmd = {}
test_error = {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    test_error[fold_num] = {}
    for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):


        o = "\', \'".join(str(x) for x in order)
        directory = "../code/baseline/cnn-tree/1/{}/0.01/20000/0.0/".format(fold_num)
        filename = "test-error-[\'{}\'].npy".format(o)


        full_path = directory + filename
        my_file = Path(full_path)
        if not my_file.exists():
            print(fold_num, o)
            o = " ".join(str(x) for x in order)
            line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 1 {} {} 0 {} {}\n".format(gid, lr, num_iterations, fold_num, o))
            print(line)
#             cmd[c] = line
            with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
                the_file.write("#!bin/bash \n")
                the_file.write(line)
            c += 1
            if c%50 == 0:
                gid += 1
            if gid == 4:
                gid = 0
                srv += 1
        else:
            test_error[fold_num][o] = np.load(full_path).item()

print(c)                               

3 dr', 'fridge', 'dw', 'mw', 'hvac
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr fridge dw mw hvac

3 dr', 'dw', 'fridge', 'hvac', 'mw
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr dw fridge hvac mw

3 dr', 'dw', 'mw', 'hvac', 'fridge
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr dw mw hvac fridge

3 dr', 'dw', 'mw', 'fridge', 'hvac
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr dw mw fridge hvac

3 dr', 'mw', 'hvac', 'fridge', 'dw
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr mw hvac fridge dw

3 dr', 'mw', 'hvac', 'dw', 'fridge
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr mw hvac dw fridge

3 dr', 'mw', 'fridge', 'hvac', 'dw
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr mw fridge hvac dw

3 dr', 'mw', 'fridge', 'dw', 'hvac
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 1 0.01 20000 0 3 dr mw fridge dw hvac

3 dr', 'mw', 'dw', 'hvac', 'fridge
CUDA_VISIBLE_DEVICES=0 python cnn-tre

4 dw', 'mw', 'dr', 'hvac', 'fridge
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 dw mw dr hvac fridge

4 dw', 'mw', 'dr', 'fridge', 'hvac
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 dw mw dr fridge hvac

4 mw', 'hvac', 'fridge', 'dr', 'dw
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 mw hvac fridge dr dw

4 mw', 'hvac', 'fridge', 'dw', 'dr
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 mw hvac fridge dw dr

4 mw', 'hvac', 'dr', 'fridge', 'dw
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 mw hvac dr fridge dw

4 mw', 'hvac', 'dr', 'dw', 'fridge
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 mw hvac dr dw fridge

4 mw', 'hvac', 'dw', 'fridge', 'dr
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 mw hvac dw fridge dr

4 mw', 'hvac', 'dw', 'dr', 'fridge
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 1 0.01 20000 0 4 mw hvac dw dr fridge

4 mw', 'fridge', 'hvac', 'dr', 'dw
CUDA_VISIBLE_DEVICES=3 python cnn-tre

In [7]:
results = {}
iters = {}
for fold_num in range(3):
    results[fold_num] = {}
    iters[fold_num] = {}
for fold_num in range(3):
    
    for order in test_error[fold_num]:
        iters[fold_num][order] = pd.DataFrame(test_error[fold_num][order]).mean(axis=0).idxmin()
    for order in test_error[fold_num]:
        results[fold_num][order] = test_error[fold_num][order][iters[fold_num][order]]
    #     print(pd.Series(test_error[fold_num][order][iters[order]]))

In [8]:
best_order = pd.DataFrame(results[0]).mean().idxmin()

In [9]:
best_order

"dw', 'dr', 'hvac', 'fridge', 'mw"

In [54]:
results[0][best_order]

{'dr': 38.77961188603765,
 'dw': 11.3265345788161,
 'fridge': 32.244577389106126,
 'hvac': 166.02465442789855,
 'mw': 7.3688055727249528}